In [1]:
class Book:
    total_books = 0  # Class variable to track the total number of books

    def __init__(self, book_id, title, author, availability=True):
        self._book_id = book_id
        self._title = title
        self._author = author
        self._availability = availability
        Book.total_books += 1  # Increment total books when a new book is added

    # Getter methods for encapsulated attributes
    @property
    def book_id(self):
        return self._book_id

    @property
    def title(self):
        return self._title

    @property
    def author(self):
        return self._author

    @property
    def availability(self):
        return self._availability

    @availability.setter
    def availability(self, status):
        self._availability = status

    @classmethod
    def get_total_books(cls):
        return cls.total_books


class User:
    total_users = 0  # Class variable to track the total number of users

    def __init__(self, user_id, name, email):
        self._user_id = user_id
        self._name = name
        self._email = email
        User.total_users += 1  # Increment total users when a new user is added

    # Getter methods for encapsulated attributes
    @property
    def user_id(self):
        return self._user_id

    @property
    def name(self):
        return self._name

    @property
    def email(self):
        return self._email

    @classmethod
    def get_total_users(cls):
        return cls.total_users

    @staticmethod
    def validate_email(email):
        # Simple validation for email (can be expanded)
        return "@" in email and "." in email


class Member(User):
    def __init__(self, user_id, name, email, library_manager):
        super().__init__(user_id, name, email)
        library_manager.add_user(self)  # Automatically add the member to the library system

    def borrow_book(self, book):
        if book.availability:
            book.availability = False
            print(f"Book '{book.title}' borrowed by {self.name}.")
        else:
            print(f"Book '{book.title}' is already borrowed and not available.")

    def return_book(self, book):
        if not book.availability:
            book.availability = True
            print(f"Book '{book.title}' returned by {self.name}.")
        else:
            print(f"Book '{book.title}' was not borrowed.")


class LibraryManager(User):
    def __init__(self, user_id, name, email):
        super().__init__(user_id, name, email)
        self._books = []  # Encapsulated book list
        self._users = []  # Encapsulated user list
        self.load_books_from_file()
        self.load_users_from_file()
        self.add_user(self)  # Automatically add the librarian to the system

    def add_book(self, book):
        self._books.append(book)
        self.save_books_to_file()
        print(f"Book '{book.title}' added to the library.")

    def remove_book(self, book_id):
        for book in self._books:
            if book.book_id == book_id:
                self._books.remove(book)
                self.save_books_to_file()
                print(f"Book '{book.title}' removed from the library.")
                return
        print(f"Book with ID '{book_id}' not found in the library.")

    def check_book_availability(self, book_id):
        for book in self._books:
            if book.book_id == book_id:
                return book.availability
        return False

    def borrow_book(self, book):
        if book.availability:
            book.availability = False
            self.save_books_to_file()
            print(f"Book '{book.title}' has been borrowed by the librarian.")
        else:
            print(f"Book '{book.title}' is not available.")

    def save_books_to_file(self):
        try:
            with open('books.txt', 'w') as f:
                for book in self._books:
                    f.write(f"{book.book_id},{book.title},{book.author},{book.availability}\n")
        except IOError:
            print("Error saving books to file.")

    def load_books_from_file(self):
        try:
            with open('books.txt', 'r') as f:
                for line in f:
                    data = line.strip().split(',')
                    book_id, title, author, availability = data
                    book = Book(book_id, title, author, availability == 'True')
                    self._books.append(book)
        except FileNotFoundError:
            print("Books file not found, starting with an empty library.")
        except IOError:
            print("Error loading books from file.")

    def save_users_to_file(self):
        try:
            with open('users.txt', 'w') as f:
                for user in self._users:
                    f.write(f"{user.user_id},{user.name},{user.email}\n")
        except IOError:
            print("Error saving users to file.")

    def load_users_from_file(self):
        try:
            with open('users.txt', 'r') as f:
                for line in f:
                    data = line.strip().split(',')
                    user_id, name, email = data
                    user = Member(user_id, name, email, self)
                    self._users.append(user)
        except FileNotFoundError:
            print("Users file not found, starting with an empty list of users.")
        except IOError:
            print("Error loading users from file.")

    def add_user(self, user):
        self._users.append(user)
        self.save_users_to_file()
        print(f"User '{user.name}' added to the library.")


# Example of usage:

# Initialize a librarian
librarian = LibraryManager(user_id="lib01", name="Librarian", email="lib@example.com")

# Initialize books
book1 = Book(book_id="bk01", title="The Catcher in the Rye", author="J.D. Salinger")
book2 = Book(book_id="bk02", title="To Kill a Mockingbird", author="Harper Lee")

# Librarian adds books to the library
librarian.add_book(book1)
librarian.add_book(book2)

# Initialize a member and add them to the library
member1 = Member(user_id="mem01", name="Alice", email="alice@example.com", library_manager=librarian)

# Member borrows a book
member1.borrow_book(book1)

# Member returns the book
member1.return_book(book1)

# Class methods to track total books and users
print(f"Total books in the system: {Book.get_total_books()}")
print(f"Total users in the system: {User.get_total_users()}")


Books file not found, starting with an empty library.
Users file not found, starting with an empty list of users.
User 'Librarian' added to the library.
Book 'The Catcher in the Rye' added to the library.
Book 'To Kill a Mockingbird' added to the library.
User 'Alice' added to the library.
Book 'The Catcher in the Rye' borrowed by Alice.
Book 'The Catcher in the Rye' returned by Alice.
Total books in the system: 2
Total users in the system: 2
